In [1]:
from openff.qcsubmit.common_structures import QCSpec, SCFProperties
from openff.qcsubmit.factories import OptimizationDatasetFactory
from openff.qcsubmit import workflow_components
import numpy as np
from qcelemental.models.results import WavefunctionProtocolEnum
from qcportal.models.common_models import DriverEnum
from openeye import oechem

In [2]:
factory = OptimizationDatasetFactory(qc_specifications={'spice_default': QCSpec(method='wb97m-d3bj', 
                                                                   basis='def2-tzvppd', 
                                                                   program='psi4', 
                                                                   spec_name='spice_default', 
                                                                   spec_description='SPICE quantum chemistry specification',
                                                                   store_wavefunction=WavefunctionProtocolEnum.orbitals_and_eigenvalues, 
                                                                   implicit_solvent=None, 
                                                                   maxiter=200, 
                                                                   scf_properties=[SCFProperties.Dipole, SCFProperties.Quadrupole, SCFProperties.WibergLowdinIndices, 
                                                                                   SCFProperties.MayerIndices, SCFProperties.MBISCharges],
                                                                   keywords=None)},
                             store_wavefunction=WavefunctionProtocolEnum.orbitals_and_eigenvalues)
factory.add_workflow_components(workflow_components.StandardConformerGenerator(max_conformers=10))
factory.dict()

{'qc_specifications': {'spice_default': {'method': 'wb97m-d3bj',
   'basis': 'def2-tzvppd',
   'program': 'psi4',
   'spec_name': 'spice_default',
   'spec_description': 'SPICE quantum chemistry specification',
   'store_wavefunction': 'orbitals_and_eigenvalues',
   'implicit_solvent': None,
   'maxiter': 200,
   'scf_properties': ['dipole',
    'quadrupole',
    'wiberg_lowdin_indices',
    'mayer_indices',
    'mbis_charges'],
   'keywords': None}},
 'driver': 'gradient',
 'priority': 'normal',
 'dataset_tags': ['openff'],
 'compute_tag': 'openff',
 'type': 'OptimizationDatasetFactory',
 'workflow': [{'type': 'StandardConformerGenerator',
   'rms_cutoff': None,
   'max_conformers': 10,
   'clear_existing': True}],
 'optimization_program': {'program': 'geometric',
  'coordsys': 'dlc',
  'enforce': 0.0,
  'epsilon': 1e-05,
  'reset': True,
  'qccnv': False,
  'molcnv': False,
  'check': 0,
  'trust': 0.1,
  'tmax': 0.3,
  'maxiter': 300,
  'convergence_set': 'GAU',
  'constraints': {}}

In [3]:
dataset = factory.create_dataset(dataset_name="SPICE Dipeptides Optimization Dataset v1.0", molecules="dipeptides.smi", tagline="QM Dataset for ML", description="Data source: https://github.com/openmm/qmdataset/tree/main/dipeptides")

Preparation                   : 100%|█████████| 677/677 [00:50<00:00, 13.50it/s]


In [4]:
confs = np.array([len(mol.conformers) for mol in dataset.molecules])

print("Number of unique molecules       ", dataset.n_molecules)
print("Number of filtered molecules     ", dataset.n_filtered)
print("Number of conformers             ", dataset.n_records)
print("Number of conformers min mean max",
      confs.min(), "{:6.2f}".format(confs.mean()), confs.max())

masses = []
for molecule in dataset.molecules:
    oemol = molecule.to_openeye()
    mass = oechem.OECalculateMolecularWeight(oemol)
    masses.append(mass)

print(f'Mean molecular weight: {np.mean(np.array(masses)):.2f}')
print(f'Max molecular weight: {np.max(np.array(masses)):.2f}')
print("Charges:", sorted(set(m.total_charge/m.total_charge.unit for m in dataset.molecules)))

Number of unique molecules        677
Number of filtered molecules      0
Number of conformers              6767
Number of conformers min mean max 7  10.00 10
Mean molecular weight: 313.72
Max molecular weight: 445.51
Charges: [-2.0, -1.0, 0.0, 1.0, 2.0]


In [5]:
dataset.visualize("dipeptides.pdf")

In [6]:
dataset.metadata.submitter = 'Josh Horton, Pavan Behara, David Dotson'

In [7]:
dataset.metadata.long_description_url = 'https://github.com/openforcefield/qca-dataset-submission/tree/master/submissions/2021-11-11-QMDataset-Dipeptide-optimization-set'

In [8]:
dataset.metadata

Metadata(submitter='Josh Horton, Pavan Behara, David Dotson', creation_date=datetime.date(2021, 11, 18), collection_type='OptimizationDataset', dataset_name='SPICE Dipeptides Optimization Dataset v1.0', short_description='QM Dataset for ML', long_description_url=HttpUrl('https://github.com/openforcefield/qca-dataset-submission/tree/master/submissions/2021-11-11-QMDataset-Dipeptide-optimization-set', scheme='https', host='github.com', tld='com', host_type='domain', path='/openforcefield/qca-dataset-submission/tree/master/submissions/2021-11-11-QMDataset-Dipeptide-optimization-set'), long_description='Data source: https://github.com/openmm/qmdataset/tree/main/dipeptides', elements={'O', 'S', 'C', 'N', 'H'})

In [9]:
dataset.export_dataset("dataset.json.bz2")